<a href="https://colab.research.google.com/github/kridtapon/KAMA-RSX-Adaptive-Strategy/blob/main/KAMA_RSX_Adaptive_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00


In [2]:
pip install --upgrade yfinance

In [7]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Kaufman Adaptive Moving Average (KAMA)
def calculate_kama(df, period):
    """
    Calculate Kaufman Adaptive Moving Average (KAMA).
    """
    return df['Close'].ewm(span=period, adjust=False).mean()

# Function to calculate ATR Ratio
def calculate_atr_ratio(df, period=14):
    """
    Calculate ATR Ratio for volatility assessment.
    """
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr / df['Close']

# Function to calculate RSX (Smoothed RSI)
def calculate_rsx(df, period=14):
    """
    Smoothed RSI (RSX) Calculation.
    """
    delta = df['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate KAMA indicators
df['KAMA_10'] = calculate_kama(df, period=10)
df['KAMA_50'] = calculate_kama(df, period=50)

# Calculate ATR Ratio
df['ATR_Ratio'] = calculate_atr_ratio(df, period=14)

# Calculate RSX
df['RSX'] = calculate_rsx(df, period=14)

# Define Squeeze Condition
df['Squeeze'] = (abs(df['KAMA_10'] - df['KAMA_50']) < df['ATR_Ratio'] * 0.5)

# Define Breakout Condition
df['Breakout_Up'] = (df['KAMA_10'] > df['KAMA_50']) & (df['RSX'] > 50)
df['Breakout_Down'] = (df['KAMA_10'] < df['KAMA_50']) & (df['RSX'] < 50)

# Define Entry and Exit signals
df['Entry'] = df['Squeeze'] | df['Breakout_Up']
df['Exit'] = (df['KAMA_10'].diff().abs() < 0.01) | df['Breakout_Down'] # Exit when KAMA flattens

# Filter data for the test period
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           439422.899142
Total Return [%]                       339.422899
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                       4647.652168
Max Drawdown [%]                        24.778758
Max Drawdown Duration                       353.0
Total Trades                                   14
Total Closed Trades                            13
Total Open Trades                               1
Open Trade PnL                       51133.940025
Win Rate [%]                            46.153846
Best Trade [%]                         136.194822
Worst Trade [%]                         -10.41989
Avg Winning Trade [%]                   38.433566
Avg Losing Trade [%]                    -4.525444
